<img src='https://i.imgur.com/RDAD11M.png' width = '200' align = 'right'>

## *DATA SCIENCE / SECTION 1 / SPRINT 1 / NOTE 2*

---

# Feature Engineering

## 🏆 학습 목표 

- Feature Engineering 의 목적을 이해 할 수 있다.
- pandas를 통해 문자열(string)을 다룰 수 있다.
- pandas를 통해 날짜와 시간 (date, time)을 다룰 수 있다.
- 데이터프레임에 `.apply()`를 사용하여 행을 수정하거나 새로 작업 할 수 있다.

---

## Feature Engineering이란

Feature Engineering 은 도메인 지식과 창의성을 바탕으로, 데이터셋에 존재하는 항목들을 재조합하여 새로운 데이터를 만드는 것입니다. 

머신러닝 모델들은 데이터에 있는 패턴을 인식하고, 해당 패턴들을 바탕으로 예측을 합니다. 

즉 모델의 더 좋은 예측을 위해서 데이터를 바탕으로 새로운 패턴을 제공하는 것이 궁극적인 목적입니다.

오늘 연습할 내용들은 이를 위해 상당히 중요한, Pandas를 사용하여 데이터프레임에 있는 열(column)을 다루는 것입니다.

데이터프레임의 각 열은 특정한 타입의 데이터를 포함하고 있습니다. 

일반적으로 쓰이는 데이터 타입을 알아보고, 주어진 데이터를 바탕으로 새로운 피처를 생성해 보도록합시다. 

## ❓ 데이터프레임이란

<img src='https://user-images.githubusercontent.com/6457691/90101634-094cd400-dd7a-11ea-8202-fff28721ba47.png' width = 600>

# Ames Iowa Housing 데이터셋:

<https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data>

In [ ]:
import pandas as pd

df = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/house-prices/house_prices_train.csv')

print(df.shape)
print(df.head())
print(df.tail())


### 각 열마다 특정한 타입의 데이터를 포함하고 있습니다. 

In [ ]:
pd.set_option('display.max_rows',10)

df.dtypes

`BedroomAbvGr` (Bedrooms Above Grade, 집 내부 지상층의 침실 수) 열은 정수형(Integer) 값을 포함하고 있습니다. 

각 열의 데이터가 의미하는 바를 알아보기 위해서는 Data Description 부분을 참조하면 좋습니다. 

In [ ]:
# `BedroomAbvGr`의 첫 10개 데이터
df['BedroomAbvGr'].head(10)

`LotFrontage` 열은 실수형(Float) 값을 포함하고 있습니다. 

In [ ]:
# `LotFrontage` 열의 첫 10개 데이터를 확인해보세요.
df['LotFrontage'].head(10)

값들이 "진짜" 실수형 값인가요?

대부분의 값들이 .0 으로 끝나고 있어서 이론적으로는 실수값이 맞지만, 실수형태로 *저장*되어야 할 이유가 있나요?

해당 열에 포함되어있는 가능한 값들을 확인해 보도록 하겠습니다. 

In [ ]:
df['LotFrontage'].value_counts(dropna = False)

결과를 보면, `LotFrontage` 열은 원래 정수형 값이지만, `NaN`으로 인해서 어쩔 수 없이 실수형으로 **형변환** 된 것으로 확인 됩니다. 

`NaN` 이란 "Not a Number"를 의미하는 것으로, Pandas에서 결측치를 취급하는 기본값입니다. 

즉 해당 셀들의 경우 LotFrontage 값이 존재 하지 않기 때문에 NaN이 기록되었다는 것을 의미합니다. 

이 경우가 바로 도메인 지식이 필요한 예시가 되는데, 주택에 관해서 배경지식을 가지고 있다면, 

`연결된 도로까지의 거리` (Linear feet of street connected to property)가 이 주택 데이터셋에서 null 혹은 비어있는 경우 어떻게 처리 해야 할까요?

또 "NaN" 혹은 "NA", "Not Applicable" 중 어떤 값으로 정하는지도 고민해봐야할 문제입니다.

그러나 한가지 기억해야할 중요한 문제는 pandas로 다루기 쉽게 한가지 형태로 결측치를 통일해야 한다는 것입니다. 

In [ ]:
import numpy as np

# NaN의 데이터 타입
type(np.NaN)

`NaN` 자체의 데이터타입은 `float`입니다. 즉 앞서 확인한 것 처럼 integer 값을 가지고 있는 열의 경우, 단 1개의 `NaN`값으로 인해 열 전체가 float로 처리된다는 것을 의미합니다. 

### 새 피쳐 생성

데이터셋의 일부를 조금 집중해서 보도록 합시다:

- `TotalBsmtSF`
- `1stFlrSF`
- `2ndFlrSF`
- `SalePrice1`


<img src='https://i.imgur.com/5kbk3yJ.png' width = '500'>

In [ ]:
# 위의 항목을 바탕으로 **작은** 데이터 프레임을 생성합니다. 
# 이는 각 column의 헤더 이름을 사용하여 할 수 있습니다. 
small_df = df[['TotalBsmtSF','1stFlrSF','2ndFlrSF','SalePrice']].copy()

small_df.head()

### 새로운 열을 생성하는 방법 

데이터프레임에서 새로운 열을 생성할때, `[ ]`를 사용하여 열에 접근합니다. (`.` 는 불가능) 

In [ ]:
# 앞의 3면적 feature의 합을 새롭게 정의 하도록 합시다. 

# [ ] 를 사용하여 새 'TotalSquareFootage' 열을 생성하세요.

small_df['TotalSquareFootage'] = small_df['TotalBsmtSF'] + small_df['1stFlrSF'] + small_df['2ndFlrSF']

small_df.head()

In [ ]:
# 이번에는 'PricePerSqFt' 라는 열을 새로 만들어보도록 하겠습니다. 
# 앞서 구한 총합을 SalePrice로 나누어 구하면 됩니다 .

small_df['PricePerSqFt'] = (small_df['SalePrice'] / small_df['TotalSquareFootage'])

이제 우리는 **작은** 데이터셋을 기준으로 새로운 2개의 피처를 생성하는데 성공했습니다. 

- **높은** `PricePerSqFt` 값이 의미하는 바에 대해서 설명 할 수 있나요?

- 반대로 **낮은** `PricePerSqFt` 값을 갖는 주택은 어떠한 의미를 가지고 있나요? 

# Pandas로 문자열 작업

## 개요

여태까지는 숫자형 (Numeric, int, float) 데이터를 작업했습니다. 이번에는 문자열(string)을 다루는 방법에 대해서 배워보겠습니다.

먼저 새로운 데이터셋을 불러오도록 합니다. 이 데이터셋은 LendingClub의 2018년 4분기에 이뤄진 대출과 관련한 데이터셋입니다. 데이터 자체가 **깔끔**하진 않기 때문에 정리를 포함하여 여러 작업을 연습 하게 될 것입니다. 

쉘의 `!wget` 명령어는 크롬과 같은 브라우저 주소창에 URL을 입력하는 것과 동일한 역할을 합니다. 즉 해당 주소의 파일을 **요청**하는데, 이번 경우에는 웹페이지가 아니라 압축된 CSV파일 이라는것에 주의하셔야합니다. 

아래의 URL을 브라우저에 붙여넣으면 자동으로 다운로드를 할 것이며 작업중인 노트북에서 `!wget` 을 사용하면 메모리에 바로 할당이 가능합니다.

### 데이터셋 불러오기

In [ ]:
# !wget https://resources.lendingclub.com/LoanStats_2018Q4.csv.zip

`!unzip` 명령어를 사용하여 압축을 풀어 csv를 다뤄야 합니다. 

In [ ]:
# !unzip LoanStats_2018Q4.csv.zip

쉘에서도 마찬가지로 `!head` 와 `!tail`을 사용하여 원본파일을 간단히 확인 해볼 수 있습니다.

In [ ]:
!head LoanStats_2018Q4.csv

In [ ]:
!tail LoanStats_2018Q4.csv

파일을 확인 했을 때, 데이터프레임으로 csv를 불러올때에 문제를 일으킬 만한 부분이 있는지 확인하십시오. 

In [ ]:
# CSV로 불러오기
df = pd.read_csv('LoanStats_2018Q4.csv', low_memory = False)

print(df.shape)
df.head()

두가지 문제가 다음 이유로 인해 발생합니다: 

1) 처음의 빈 부분은 column의 헤더를 지정하지 않았기 떄문에 생깁니다. 
2) 맨 마지막의 두 빈 줄은 footer를 지정하지 않았기 때문에 생깁니다. 

In [ ]:
# 1)번 문제는 'skiprows' 파라미터를 이용하여 수정 할 수 있습니다.
df = pd.read_csv('LoanStats_2018Q4.csv', skiprows = 1, low_memory = False)

print(df.shape)
df.head()

각 열의 NaN값을 확인 하면 마지막 두 줄이 전부 비어있다는 것을 확인 할 수 있습니다.

In [ ]:
# 각 열 별로 null의 수를 세어 정렬합니다. 
df.isnull().sum().sort_values()

In [ ]:
# 2) 번 문제는 skipfooter 파라미터를 이용하여 수정 할 수 있습니다. 
df = pd.read_csv('LoanStats_2018Q4.csv', skiprows = 1, skipfooter = 2, engine = 'python')
print(df.shape)
df.head()

In [ ]:
df.isnull().sum().sort_values()

조금 더 나은 작업을 위해, 모든 값이 NaN으로 이루어진 열들을 제거하도록 하겠습니다. 

LendingClub에서 전부 비어있는 항목을 데이터셋에 추가한 이유를 한번 고민해보시면 좋을 것 같습니다.

In [ ]:
df = df.drop(['url','member_id','desc','id'], axis = 1)
print(df.shape)
df.head()

### `int_rate` 열 정리

일반적으로 머신러닝 모델링을 위한 데이터셋에서는 문자열로 이루어진 값은 사용하지 않습니다. (이는 매우 복잡한 이슈입니다.)

구체적인 예시로, `int_rate` 항목을 통해 숫자이지만 숫자로 구성되지 않은 예시를 알아보도록 하겠습니다. 처음 10개의 값을 확인해보세요.

In [ ]:
# int_rate의 첫 10개 값을 확인합니다.
df['int_rate'][:10]

이 항목에서 해결해야할 이슈는 다음과 같습니다:

- 숫자로 바뀌어야할 문자열이 존재함
- `%` 가 숫자에 같이 포함되어 있음
- 문자열의 처음에 공백이 포함되어 있음

<img src='https://user-images.githubusercontent.com/6457691/90108851-e1fc0400-dd85-11ea-8db0-eabd587f98f8.png' width = 400>

이 문제들을 한번에 해결하기 보단 한단계씩 접근하도록 하겠습니다.

[python strip](https://www.w3schools.com/python/ref_string_strip.asp)

In [ ]:
# 한개의 문자열 수정을 연습하도록 합니다.
int_rate = ' 14.17%' # 공백 과 % 에 주의하세요

In [ ]:
int_rate.strip()

In [ ]:
int_rate.strip('%')

In [ ]:
int_rate.strip().strip('%')

In [ ]:
type(int_rate.strip().strip('%'))

In [ ]:
# 문자열을 실수형으로 "형변환" ("Cast")합니다.
float(int_rate.strip().strip('%'))

In [ ]:
type(float(int_rate.strip().strip('%')))

### 이제 여러번 반복 할 수 있는 함수를 만들어보도록 하겠습니다.

[python function](https://www.w3schools.com/python/python_functions.asp)

In [ ]:
# 입력된 문자열에 대해서 같은 작업을 하는 함수 작성
def int_rate_to_float(cell_contents):
    return float(cell_contents.strip().strip('%'))

In [ ]:
# 예시 데이터를 바탕으로 함수를 테스트
int_rate_to_float(int_rate)

In [ ]:
# 데이터 타입을 확인하십시오
type(int_rate_to_float(int_rate))

### 열의 모든 데이터에 함수를 적용합니다.

In [ ]:
df['int_rate_float'] = df['int_rate'].apply(int_rate_to_float)

df.head()

In [ ]:
# 데이터셋의 마지막 5개 열의 타입을 확인

df.dtypes[-5:]

# `.apply()`를 이용하여 열을 수정 및 생성하기

## 개요

[pandas apply](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html)

이전에 이미 `.apply()`를 사용하여 행의 데이터 정리 하는 예시를 다뤄봤습니다. 이번에는 조금 더 복잡한 과정을 적용하는 연습을 해보도록 하겠습니다. 

염두해야 할 것은, 이번 주제의 목표는 **개개의** 셀에 대해서 `.apply()`를 통해 적용하는 함수를 작성하는 것입니다. 

"Employment Title" (emp_title) 열을 예시로 연습해보도록 하겠습니다. 

## 따라하세요.

우선으로 해야 할 것은, 문제가 무엇이고 그 문제 개선을 위해서 어떤 것을 해야하는지를 인지하는 것입니다. 

In [ ]:
# 상위 20개의 employment titles 확인
df['emp_title'].value_counts(dropna=False, ascending=False)[:20]

In [ ]:
# 얼마나 많은 종류의 항목이 있는지 확인
len(df['emp_title'].value_counts())

In [ ]:
# employment_title이 null (NaN) 인 경우 확인
df['emp_title'].isnull().sum()

직업이 주어 지지 않는 경우는 어떠한 경우인지 생각해봅시다.

In [ ]:
# 예시 데이터 생성
examples = ['owner', 'Supervisor', 'Project Manager ', np.NaN]

# 이번 데이터 정리의 목적은 주어진 직함에 대해서 첫 글자를 대문자로 바꾸는 것입니다. (owner -> Owner)

In [ ]:
# 함수 작성

def clean_title(title):
    if isinstance(title, str):
        return title.strip().title()
    else:
        return "Unknown"
    
for example in examples:
    print(clean_title(example))

In [ ]:
# list 표현을 사용하여, list 내부의 아이템에 대해서 함수를 적용 할 수 있습니다.

[clean_title(example) for example in examples]

In [ ]:
# 작동하는 함수를 생성했으니, 주어진 열에 대해서 적용 해보도록 합시다.
# 덮어쓰기에 주의하십시오.

df['emp_title'] = df['emp_title'].apply(clean_title)
df.head()


함수 적용 후, 앞에서 사용 했던 코드를 통해 잘 적용 되었는지 확인합니다.

In [ ]:
# 상위 20개의 employment titles 확인
df['emp_title'].value_counts(dropna = False, ascending = False)[:20]

In [ ]:
# 얼마나 많은 종류의 항목이 있는지 확인
len(df['emp_title'].value_counts())

In [ ]:
# employment_title이 null (NaN) 인 경우 확인
df['emp_title'].isnull().sum()

# Pandas를 통해 날짜, 시간 데이터 다루기

## 개요

Pandas 에는 문자열로 이뤄진 날짜와 시간을 다루기 위한 자체적인 데이터 타입이 있습니다. 

이번 섹션에서는 날짜(문자열)를 datetime 이라는 오브젝트로 바꾸는 방법과 `.dt`라는 형식을 통해서 사용하는 방법을 다루도록 하겠습니다.

### 날짜 다루기

pandas 메뉴얼
- [to_datetime](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.to_datetime.html)
- [Time/Date Components](https://pandas.pydata.org/pandas-docs/stable/timeseries.html#time-date-components) "`.dt` 를 사용하여 해당 속성에 접근 할 수 있습니다."

많은 경우, 날짜를 다루는 열은 `_d` 라는 단어를 사용해 표기하고 있습니다. 

이 후 List를 사용하여 출력 하는 것까지 해 보도록 하겠습니다.

In [ ]:
[col for col in df if col.endswith('_d')]

위의 코드는 데이터의 열 가운데 `~~_d` 형태를 갖는 열들을 찾습니다.

In [ ]:
df['issue_d'][:10]

일반적인 datetime 형태를 위한 문자열은 %m-%y (월,연 각각) 형태를 띄고 있기 때문에, pandas를 사용하여 해당 패턴을 감지하고 datetime 오브젝트로 변환 할 수 있습니다.

In [ ]:
df['issue_d'] = pd.to_datetime(df['issue_d'], infer_datetime_format = True)

df.dtypes[:15]

위의 코드를 통해서 `issue_d` 열이 `datetime` 오브젝트 형태로 바뀌었음을 확인 할 수 있습니다.

그렇다면 구체적으로 datetime 오브젝트가 의미하는 것이 무엇인지 조금 더 들여다보도록 하겠습니다.

In [ ]:
df['issue_d'].iloc[0] # iloc 는 integer location을 의미합니다.

월과 연도에 해당하는 부분이 문자열에서 어떻게 바뀌었는지 확인 할 수 있습니다.

In [ ]:
df['issue_d'].head().values

이제 주어진 datetime 오브젝트에 대해서 특정 내용을 선택하기 위해서, `.dt`를 사용할 수 있습니다. 예시로 `issue_d`열의 모든 데이터에 대해서 연도와 월 데이터만 따로 추출해보도록 하겠습니다.

In [ ]:
df['issue_d'].dt.year # .dt datetime object

In [ ]:
df['issue_d'].dt.month

이제 어렵지 않습니다. 다음은 이를 출력하는 대신, 각각의 연도와 월의 값을 데이터프레임에 새로운 열로 추가해보도록 하겠습니다. 화면을 오른쪽으로 스크롤하여, 새롭게 열이 추가가 된 것을 확인하세요.

In [ ]:
df['issue_year'] = df['issue_d'].dt.year
df['issue_month'] = df['issue_d'].dt.month

df.head()

데이터셋 자체가 2018년도의 4분기만을 다루고 있기 때문에 `issue_d` 데이터 자체는 크게 의미가 있지 않습니다. 대신 `earliest_cr_line` 라는 문자열로 이루어졌지만 마찬가지로 datetime 형태로 바꿀 수 있는 열을 조사해보록 하겠습니다.

이번에 하게 될 일은 `days_from_earliest_credit_to_issue` 라는 이름을 갖는 새로운 열을 생성하는 것입니다. 

열의 헤더의 길이가 깁니다만, 이러한 이름들을 의미를 이해 할 수 있게 정하는 것은 상당히 중요한 문제이며 앞으로도 많이 고민을 해봐야 할 것입니다. 

In [ ]:
df['earliest_cr_line'].head()

In [ ]:
df['earliest_cr_line'] = pd.to_datetime(df['earliest_cr_line'], infer_datetime_format = True)

print(df['earliest_cr_line'].head())

판다스의 datetime 객체는 상당히 똑똑해서 단순히 빼기 연산자 `-`를 사용하는 것만으로도 두 날짜 사이의 시간을 계산할 수 있습니다.


In [ ]:
print(df['issue_d'].head())
print(df['earliest_cr_line'].head())

In [ ]:
df['days_from_ealiest_credit_to_issue'] = (df['issue_d'] - df['earliest_cr_line']).dt.days # 날짜의 차이를 구할 수 있음

In [ ]:
df['days_from_ealiest_credit_to_issue'].head()

2018년 4분기의 가장 오래된 기록을 확인 할 수 있나요?

In [ ]:
df['days_from_ealiest_credit_to_issue'].describe()

(TMI) 25,171일은 ~ 68.96년 입니다